<h1>国勢調査データに基づいた分類モデルの構築と使用</h1>

BigQuery ML は、[ロジスティック回帰](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#model_type)モデルタイプでの教師あり学習をサポートしています。2 つのカテゴリのどちらに値が分類されるかを予測するには 2 項ロジスティック回帰モデルタイプを使用し、複数のカテゴリのうちのどれに値が分類されるかを予測するには多項回帰モデルタイプを使用します。これらはデータを 2 つ以上のカテゴリに分類するため、分類問題と呼ばれます。

このチュートリアルでは、BigQuery ML の 2 項ロジスティック回帰モデルを使用して、[米国国勢調査データセット](https://cloud.google.com/bigquery/public-data)の回答者の所得階層を予測します。このデータセットには、2000 年と 2010 年の米国居住者のユーザー属性と所得情報が含まれています。このデータには、就業状況、職種、教育レベル、所得の情報が含まれています。

<h2>目標</h2>

このチュートリアルでは、次のタスクを行います。
- ロジスティック回帰モデルを作成する。
- ロジスティック回帰モデルを評価する。
- ロジスティック回帰モデルを使用して予測を行う。

<h2>はじめに</h2>

機械学習の一般的な問題は、ラベルと呼ばれる 2 つのタイプのいずれかにデータを分類することです。たとえば、小売業者は特定の顧客が新しい製品を購入するかどうかを、その顧客に関するその他の情報に基づいて予測したい場合があります。この場合、2 つのラベルは「購入する」と「購入しない」になります。データセットを構築する際にはこれらのラベルを表す列を 1 つ用意します。このような [2 項ロジスティック回帰](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#model_type)モデルのトレーニングに使用できるデータとして、顧客の場所、以前の購入、報告された顧客の好みなどがあります。

このチュートリアルでは、BigQuery ML を使用して、米国国勢調査の回答者のユーザー属性に基づいて、その回答者の所得が 2 つの範囲のどちらに分類されるかを予測する 2 項ロジスティック回帰モデルを作成します。

<h2>ステップ 1: データセットを作成する</h2>

In [ ]:
!bq mk --location us census

<h2>ステップ 2: データを確認する</h2>

次に、データセットを確認して、ロジスティック回帰モデルのトレーニング データとして使用する列を特定します。データセットから行を返すには、標準 SQL クエリを使用できます。

次のクエリでは、米国国勢調査データセットから 100 行が返されます。

In [ ]:
%%bigquery res
SELECT
  *
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`
LIMIT
  100;

In [ ]:
res.head(10)

<h2>ステップ 3: トレーニング データを選択する</h2>

次に、ロジスティック回帰モデルのトレーニングに使用するデータを選択します。このチュートリアルでは、次の属性に基づいて国勢調査回答者の所得を予測します。

- 年齢
- 業務のタイプ
- 居住国
- 婚姻状況
- 教育水準
- 職業
- 人種
- 週あたりの労働時間

次のクエリでは、トレーニング データをまとめるビューを作成します。このビューは、チュートリアルの後半で `CREATE MODEL` ステートメントで使われます。

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW
  `census.input_view` AS
SELECT
  age,
  workclass,
  native_country,
  marital_status,
  education_num,
  occupation,
  race,
  hours_per_week,
  income_bracket,
  CASE
    WHEN MOD(functional_weight, 10) < 8 THEN 'training'
    WHEN MOD(functional_weight, 10) = 8 THEN 'evaluation'
    WHEN MOD(functional_weight, 10) = 9 THEN 'prediction'
  END AS dataframe
FROM
  `bigquery-public-data.ml_datasets.census_adult_income`

<h3>クエリの詳細</h3>

このクエリは、回答者の教育水準を表す `education_num` や、回答者が行う業務のタイプを表す `workclass` など、国勢調査回答者に関するデータを抽出します<br>
このクエリでは、データが重複するカテゴリは除外しています。たとえば、`census_adult_income` テーブルの列 `education` と `education_num` は同じデータを異なる形式で表しているため、このクエリでは `education` 列を除外しています。<br>
`dataframe` 列では、除外された `functional_weight` 列を使用して、データソースの 80% にトレーニング用としてラベルを付け、残りのデータを評価用と予測用として予約しています。クエリでこれらの列を含むビューを作成し、後でそれを使用してトレーニングと予測を行えるようにします。

In [ ]:
%%bigquery
SELECT
 column_name, data_type 
FROM census.INFORMATION_SCHEMA.COLUMNS
  WHERE table_name = 'input_view'

<h2>ステップ 4: ロジスティック回帰モデルを作成する</h2>

トレーニング データを確認したら、次のステップでは、そのデータを使用してロジスティック回帰モデルを作成します。

[CREATE MODEL](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create) ステートメントを `'LOGISTIC_REG'` オプション付きで使用することで、ロジスティック回帰モデルの作成とトレーニングが行えます。次のクエリでは、`CREATE MODEL` ステートメントを使用して、前のクエリで作成したビューで新しい 2 項ロジスティック回帰モデルをトレーニングしています。

In [ ]:
%%bigquery
CREATE OR REPLACE MODEL
  `census.census_model`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    input_label_cols=['income_bracket']
  ) AS
SELECT
  *
FROM
  `census.input_view`
WHERE
  dataframe = 'training'

<h3>クエリの詳細</h3>

`CREATE MODEL`ステートメントは、`SELECT` ステートメントのトレーニング データを使用してモデルをトレーニングしています。

`OPTIONS` 句は、モデルタイプとトレーニング オプションを指定します。ここでは、`LOGISTIC_REG` オプションでロジスティック回帰モデルタイプを指定しています。2 項ロジスティック回帰モデルか多項ロジスティック回帰モデルかを指定する必要はありません。BigQuery ML はラベル列の一意の値の数に基づいてどちらをトレーニングするかを判断できます。

`input_label_cols` オプションは、SELECT ステートメントでラベル列として使用する列を指定します。ここでは、ラベル列は income_bracket であるため、モデルは他の行の値に基づいて、income_bracket の 2 つの値のどちらに分類される可能性が高いかを学習します。

`'auto_class_weights=TRUE'` オプションは、トレーニング データ内のクラスラベルのバランスをとります。デフォルトでは、トレーニング データは重み付けされません。トレーニング データ内のラベルが不均衡である場合、モデルは最も出現回数の多いラベルクラスをより重視して予測するように学習することがあります。この場合、データセット内の回答者の大多数は低い方の所得階層に属します。このため、低い方の所得階層を過度に重視して予測するモデルになる可能性があります。クラスの重みは、各クラスの頻度に反比例した重みを計算して、クラスラベルのバランスをとります。

`SELECT` ステートメントは、ステップ 2 で作成したビューに対してクエリを実行します。このビューには、モデルをトレーニングするための特徴データの列のみが含まれます。WHERE 句は `input_view` の行をフィルタして、トレーニング データフレームに属する行のみがトレーニング データに含まれるようにします。

<h2>ステップ 5: ML.EVALUATE 関数を使用してモデルを評価する。</h2>

モデルを作成したら、[`ML.EVALUATE`](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-evaluate) 関数を使用してモデルの性能を評価します。ML.EVALUATE 関数は、実際のデータに対する予測値を評価します。

モデルの評価に使用するクエリは次のとおりです。

In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE (MODEL `census.census_model`,
    (
    SELECT
      *
    FROM
      `census.input_view`
    WHERE
      dataframe = 'evaluation'
    )
  )

ロジスティック回帰を使用しているため、結果には次の列が含まれます。

- precision
- recall
- accuracy
- f1_score
- log_loss
- roc_auc

モデルの精度は、正確に予測された値と予測全体の比率を表します。精度が .80 の場合、予測の 80% が正しいことを意味します。

<h2>ステップ 6: ML.PREDICT 関数を使用して所得階層を予測する</h2>

特定の回答者が属する所得階層を特定するには、[ML.PREDICT](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-predict) 関数を使用します。次のクエリは、`prediction` データフレーム内のすべての回答者の所得階層を予測します。

In [ ]:
%%bigquery predict
SELECT
  * except(predicted_income_bracket_probs)
FROM
  ML.PREDICT (MODEL `census.census_model`,
    (
    SELECT
      *
    FROM
      `census.input_view`
    WHERE
      dataframe = 'prediction'
     )
  )

In [ ]:
predict.head(10)

`predicted_income_bracket`は `income_bracket` の予測値です。



---

## ラボ: クレジットカードの不正利用を予測する分類モデルを作成する

`bigquery-public-data.ml_datasets.ulb_fraud_detection` データを使用して、クレジットカードの取引が詐欺であるかを分類するモデルを作成してください。<br>

In [ ]:
%%bigquery df
SELECT
  *,
  CASE
    WHEN MOD(ABS(FARM_FINGERPRINT(CAST(Time AS STRING))), 10) < 8 THEN 'training'
    WHEN MOD(ABS(FARM_FINGERPRINT(CAST(Time AS STRING))), 10) = 8 THEN 'evaluation'
    WHEN MOD(ABS(FARM_FINGERPRINT(CAST(Time AS STRING))), 10) = 9 THEN 'prediction'
  END AS dataframe
FROM
  `bigquery-public-data.ml_datasets.ulb_fraud_detection`

In [ ]:
df

### モデルを作成する

### モデルの統計情報を取得する

### モデルを評価する



Except as otherwise noted, the content of this page is licensed under the Creative Commons Attribution 4.0 License, and code samples are licensed under the Apache 2.0 License. For details, see the Google Developers Site Policies. Java is a registered trademark of Oracle and/or its affiliates.

Source: [https://cloud.google.com/bigquery-ml/docs/logistic-regression-prediction](https://cloud.google.com/bigquery-ml/docs/logistic-regression-prediction)